In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import GridSearchCV
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

In [32]:
df = pd.read_csv('../../dataset/features.csv')

In [33]:
import ast

feature_cols = [
    'mfcc_mean',
    'mfcc_delta_mean',
    'mfcc_delta2_mean',
    'mel_spec_db_mean',
]

for col in feature_cols:
    df[col] = df[col].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) else x)
    df[col] = df[col].apply(lambda x: sum(x) / len(x) if isinstance(x, list) else x)


In [34]:
idx = df[df['Emotion levels'] == "X"].index
df = df.drop(idx)
df.drop(columns=['Location'], inplace=True)

In [35]:
df['Gender'] = df['Gender'].astype("category").cat.codes

In [36]:
df = pd.get_dummies(df, columns=['Emotion levels'], drop_first=True)
dummy_cols = [col for col in df.columns if col.startswith('Emotion levels_')]
df[dummy_cols] = df[dummy_cols].astype(int)

In [37]:
df

,Gender,Age,Emotion,mfcc_mean,mfcc_delta_mean,mfcc_delta2_mean,mel_spec_db_mean,spectral_centroid_mean,zcr_mean,Emotion levels_LO,Emotion levels_MD,Emotion levels_XX
0,1,42,ANG,-9.889783,-0.026791,-0.032141,-56.827071,1179.349322,0.043249,0,0,1
1,1,29,HAP,-14.319490,-0.008121,0.001389,-53.707450,1500.493431,0.070299,0,0,1
2,0,52,ANG,-10.598191,0.000172,-0.007669,-61.971771,1761.658971,0.085656,0,1,0
3,1,62,HAP,-13.321998,-0.011745,-0.008918,-54.003731,1117.330547,0.036856,1,0,0
4,0,33,ANG,-10.971943,0.006647,-0.005334,-57.965145,1439.093152,0.064631,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...
7437,0,20,FEA,-12.769634,0.024217,0.023551,-53.149645,1733.797389,0.094620,0,0,1
7438,0,21,NEU,-16.304217,-0.002686,-0.000699,-51.632801,1312.391145,0.051037,0,0,1
7439,1,45,ANG,-10.881321,0.005241,0.017308,-52.594525,1825.146631,0.092624,1,0,0
7440,1,32,NEU,-11.946458,0.008935,-0.004965,-51.789469,1140.460215,0.037141,0,0,1


In [38]:
df.columns

Index(['Gender', 'Age', 'Emotion', 'mfcc_mean', 'mfcc_delta_mean',
       'mfcc_delta2_mean', 'mel_spec_db_mean', 'spectral_centroid_mean',
       'zcr_mean', 'Emotion levels_LO', 'Emotion levels_MD',
       'Emotion levels_XX'],
      dtype='object')

In [46]:
feature_cols = df.columns
feature_cols = feature_cols.drop('Emotion')
print(feature_cols)

Index(['Gender', 'Age', 'mfcc_mean', 'mfcc_delta_mean', 'mfcc_delta2_mean',
       'mel_spec_db_mean', 'spectral_centroid_mean', 'zcr_mean',
       'Emotion levels_LO', 'Emotion levels_MD', 'Emotion levels_XX'],
      dtype='object')


In [47]:
X = df[feature_cols]
y = df['Emotion']

In [48]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [49]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [50]:
n_neighbors = np.arange(1,51)
parameters = {'n_neighbors':n_neighbors, 'metric':('hamming', 'euclidean', 'manhattan')}
classifier = KNeighborsClassifier()
clf = GridSearchCV(classifier, parameters, scoring='accuracy', cv=5, verbose=3)
clf.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 150 candidates, totalling 750 fits
[CV 1/5] END .....metric=hamming, n_neighbors=1;, score=0.148 total time=   0.2s
[CV 2/5] END .....metric=hamming, n_neighbors=1;, score=0.146 total time=   0.2s
[CV 3/5] END .....metric=hamming, n_neighbors=1;, score=0.149 total time=   0.2s
[CV 4/5] END .....metric=hamming, n_neighbors=1;, score=0.147 total time=   0.2s
[CV 5/5] END .....metric=hamming, n_neighbors=1;, score=0.151 total time=   0.3s
[CV 1/5] END .....metric=hamming, n_neighbors=2;, score=0.137 total time=   0.3s
[CV 2/5] END .....metric=hamming, n_neighbors=2;, score=0.142 total time=   0.1s
[CV 3/5] END .....metric=hamming, n_neighbors=2;, score=0.144 total time=   0.1s
[CV 4/5] END .....metric=hamming, n_neighbors=2;, score=0.142 total time=   0.1s
[CV 5/5] END .....metric=hamming, n_neighbors=2;, score=0.149 total time=   0.2s
[CV 1/5] END .....metric=hamming, n_neighbors=3;, score=0.139 total time=   0.2s
[CV 2/5] END .....metric=hamming, n_neighbors=

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ('hamming', 'euclidean', 'manhattan'),
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])},
             scoring='accuracy', verbose=3)

In [51]:
print("best score: ", clf.best_score_)
print("best hyperparameters: ", clf.best_params_)

best score:  0.35383189043879515
best hyperparameters:  {'metric': 'manhattan', 'n_neighbors': np.int64(37)}


In [52]:
knn = KNeighborsClassifier(n_neighbors=37, metric='manhattan')
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(metric='manhattan', n_neighbors=37)

In [53]:
y_pred = knn.predict(X_test_scaled)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[177  23   7  36  10   1]
 [ 36  56  23  41  51  47]
 [ 54  31  39  26  57  48]
 [ 66  33  33  52  51  19]
 [  8  31  24  38  75  42]
 [  8  37  22   9  54 124]]
              precision    recall  f1-score   support

         ANG       0.51      0.70      0.59       254
         DIS       0.27      0.22      0.24       254
         FEA       0.26      0.15      0.19       255
         HAP       0.26      0.20      0.23       254
         NEU       0.25      0.34      0.29       218
         SAD       0.44      0.49      0.46       254

    accuracy                           0.35      1489
   macro avg       0.33      0.35      0.33      1489
weighted avg       0.33      0.35      0.33      1489



In [54]:
n_components = df['Emotion'].nunique() - 1
lda = LinearDiscriminantAnalysis(n_components=n_components)
X_train_lda = lda.fit_transform(X_train_scaled, y_train)
X_test_lda = lda.transform(X_test_scaled)

In [64]:
print(f'Original number of features: {X_train.shape[1]}')
print(f'Number of features after LDA: {X_train_lda.shape[1]}')

Original number of features: 11
Number of features after LDA: 5


In [65]:
n_neighbors = np.arange(1,51)
parameters = {'n_neighbors':n_neighbors, 'metric':('hamming', 'euclidean', 'manhattan')}
classifier = KNeighborsClassifier()
clf = GridSearchCV(classifier, parameters, scoring='accuracy', cv=5, verbose=3)
clf.fit(X_train_lda, y_train)

Fitting 5 folds for each of 150 candidates, totalling 750 fits
[CV 1/5] END .....metric=hamming, n_neighbors=1;, score=0.171 total time=   0.2s
[CV 2/5] END .....metric=hamming, n_neighbors=1;, score=0.171 total time=   0.2s
[CV 3/5] END .....metric=hamming, n_neighbors=1;, score=0.171 total time=   0.1s
[CV 4/5] END .....metric=hamming, n_neighbors=1;, score=0.171 total time=   0.2s
[CV 5/5] END .....metric=hamming, n_neighbors=1;, score=0.171 total time=   0.1s
[CV 1/5] END .....metric=hamming, n_neighbors=2;, score=0.171 total time=   0.1s
[CV 2/5] END .....metric=hamming, n_neighbors=2;, score=0.170 total time=   0.1s
[CV 3/5] END .....metric=hamming, n_neighbors=2;, score=0.171 total time=   0.2s
[CV 4/5] END .....metric=hamming, n_neighbors=2;, score=0.171 total time=   0.1s
[CV 5/5] END .....metric=hamming, n_neighbors=2;, score=0.171 total time=   0.1s
[CV 1/5] END .....metric=hamming, n_neighbors=3;, score=0.171 total time=   0.1s
[CV 2/5] END .....metric=hamming, n_neighbors=

GridSearchCV(cv=5, estimator=KNeighborsClassifier(),
             param_grid={'metric': ('hamming', 'euclidean', 'manhattan'),
                         'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50])},
             scoring='accuracy', verbose=3)

In [66]:
print("best score: ", clf.best_score_)
print("best hyperparameters: ", clf.best_params_)

best score:  0.3707979312631854
best hyperparameters:  {'metric': 'euclidean', 'n_neighbors': np.int64(49)}


In [67]:
knn = KNeighborsClassifier(n_neighbors=49, metric='manhattan')
knn.fit(X_train_scaled, y_train)

KNeighborsClassifier(metric='manhattan', n_neighbors=49)

In [68]:
y_pred = knn.predict(X_test_scaled)

print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))

[[176  24   8  36   9   1]
 [ 36  59  26  36  50  47]
 [ 57  30  27  32  58  51]
 [ 65  36  25  56  54  18]
 [  7  34  19  37  78  43]
 [  7  38  21  10  59 119]]
              precision    recall  f1-score   support

         ANG       0.51      0.69      0.58       254
         DIS       0.27      0.23      0.25       254
         FEA       0.21      0.11      0.14       255
         HAP       0.27      0.22      0.24       254
         NEU       0.25      0.36      0.30       218
         SAD       0.43      0.47      0.45       254

    accuracy                           0.35      1489
   macro avg       0.32      0.35      0.33      1489
weighted avg       0.32      0.35      0.33      1489

